In [10]:
import torch
import torchvision
from torchvision import transforms 
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import sklearn
import numpy
import pandas
import matplotlib
import os
import tqdm
from tqdm import tqdm

# -------------------------------------------------------------------
# Paths
# -------------------------------------------------------------------


cwd = os.getcwd()

if os.path.isdir(os.path.join(cwd, 'data', 'training')):
    project_root = cwd
elif os.path.isdir(os.path.join(cwd, '..', 'data', 'training')):
    project_root = os.path.abspath(os.path.join(cwd, '..'))
else:
    raise RuntimeError(
        "Non riesco a trovare la cartella 'data/training' né in "
        f"{cwd} né in {os.path.abspath(os.path.join(cwd,'..'))}"
    )

root_dir_train = os.path.join(project_root, 'data', 'training')
root_dir_test  = os.path.join(project_root, 'data', 'test')


# -------------------------------------------------------------------
# 1) Data loader: train / val / test
# -------------------------------------------------------------------
def get_data(
    batch_size,
    test_batch_size=16,
    num_workers=2,
    mean=None,
    std=None,
    num_train_samples=None
):
    """
    Returns:
      train_loader, val_loader, test_loader
    Computes mean/std on the training set if not supplied.
    """
    target_size = (224, 224)

    # 1a) compute mean/std if needed
    if mean is None or std is None:
        tmp_tf = transforms.Compose([
            transforms.Resize(target_size),
            transforms.ToTensor()
        ])
        tmp_ds = ImageFolder(root_dir_train, transform=tmp_tf)
        imgs = torch.stack([img for img, _ in tmp_ds], dim=0)
        mean = float(imgs.mean())
        std  = float(imgs.std())

    # 1b) full transforms
    tf = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean], std=[std])
    ])

    # 1c) datasets
    full_train_ds = ImageFolder(root_dir_train, transform=tf)
    test_ds       = ImageFolder(root_dir_test,  transform=tf)

    # 1d) train / val split
    N = len(full_train_ds)
    train_N = (int(N * 0.7) if num_train_samples is None else num_train_samples)
    val_N   = N - train_N
    train_ds, val_ds = torch.utils.data.random_split(full_train_ds, [train_N, val_N])

    # 1e) loaders
    train_loader = DataLoader(train_ds, batch_size=batch_size,       shuffle=True,  num_workers=num_workers)
    val_loader   = DataLoader(val_ds,   batch_size=test_batch_size,  shuffle=False, num_workers=num_workers)
    test_loader  = DataLoader(test_ds,  batch_size=test_batch_size,  shuffle=False, num_workers=num_workers)

    return train_loader, val_loader, test_loader

# -------------------------------------------------------------------
# 2) Build GoogLeNet feature extractor
# -------------------------------------------------------------------
def build_googlenet_extractor(device='cuda'):
    model = torchvision.models.googlenet(pretrained=True,
                                         aux_logits=False,
                                         transform_input=False)
    # drop final FC layer
    feat_ext = torch.nn.Sequential(*list(model.children())[:-1]).to(device)
    feat_ext.eval()
    return feat_ext

# -------------------------------------------------------------------
# 3) Extract embeddings from a loader
# -------------------------------------------------------------------
@torch.no_grad()
def extract_embeddings(loader, model, device='cuda'):
    embs = []
    for imgs, _ in loader:
        imgs = imgs.to(device)
        feats = model(imgs)               # [B, 1024, 1, 1]
        feats = feats.view(feats.size(0), -1)
        embs.append(feats.cpu())
    return torch.cat(embs, dim=0)         # [N, 1024]

# -------------------------------------------------------------------
# 4) Top-k retrieval within the same set
# -------------------------------------------------------------------
def retrieve_topk(query_embs, gallery_embs, query_paths, gallery_paths, k=5):
    # normalize for cosine
    query_embs   = F.normalize(query_embs, dim=1)
    gallery_embs = F.normalize(gallery_embs, dim=1)

    # similarity matrix
    sim = query_embs @ gallery_embs.t()      # [Q, G]

    # get k+1 highest so we can skip self-match at index 0
    topk = sim.topk(k + 1, dim=1, largest=True)[1]  # [Q, k+1]

    results = []
    for qi, idxs in enumerate(topk):
        # drop the first idx (self)
        neigh = idxs.tolist()[1 : k+1]
        qpath = query_paths[qi][0]
        retrieved = [gallery_paths[i][0] for i in neigh]
        results.append({
            'query':    os.path.basename(qpath),
            'retrieved': [os.path.basename(p) for p in retrieved]
        })
    return results

# -------------------------------------------------------------------
# 5) Main script
# -------------------------------------------------------------------
if __name__ == '__main__':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # a) load data
    train_loader, val_loader, test_loader = get_data(
        batch_size=16,
        test_batch_size=32,
        num_workers=2
    )

    # b) build feature extractor
    feat_ext = build_googlenet_extractor(device)

    # c) extract embeddings on test set
    test_embs = extract_embeddings(test_loader, feat_ext, device)

    # d) get file paths for retrieval
    test_paths = test_loader.dataset.imgs    # list of (filepath, class_idx)

    # e) retrieve top-10 similar images within the test set
    topk_results = retrieve_topk(
        test_embs, test_embs,
        test_paths, test_paths,
        k=10
    )

    # f) output JSON
    import json
    print(json.dumps(topk_results, indent=2))


c:\Users\MyPc\anaconda3\envs\ML_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\MyPc\anaconda3\envs\ML_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ValueError: The parameter 'aux_logits' expected value True but got False instead.